In [ ]:
! pip install torch torchvision torchaudio

In [ ]:
! pip install matplotlib

In [ ]:
! pip install gradio

In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [26]:
# Define transformations for the dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load the MNIST dataset
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=1000, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=1000, shuffle=False)

In [27]:
class MNISTNet(nn.Module):
    def __init__(self):
        super(MNISTNet, self).__init__()
        self.fc1 = nn.Linear(784, 150)
        self.fc2 = nn.Linear(150, 200)
        self.fc3 = nn.Linear(200, 100)
        self.fc4 = nn.Linear(100, 500)
        self.fc5 = nn.Linear(500, 10)
    
    def forward(self, x):
        x = x.view(-1, 784)  # Flatten the input
        x = torch.relu(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        x = torch.tanh(self.fc3(x))
        x = torch.tanh(self.fc4(x))
        x = self.fc5(x)
        return x

model = MNISTNet()

In [28]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [29]:
num_epochs = 5

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 10 == 9:  # Print every 10 batches
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 10:.3f}')
            running_loss = 0.0

print('Finished Training')

[Epoch 1, Batch 10] loss: 1.894
[Epoch 1, Batch 20] loss: 0.949
[Epoch 1, Batch 30] loss: 0.625
[Epoch 1, Batch 40] loss: 0.458
[Epoch 1, Batch 50] loss: 0.385
[Epoch 1, Batch 60] loss: 0.353
[Epoch 2, Batch 10] loss: 0.312
[Epoch 2, Batch 20] loss: 0.279
[Epoch 2, Batch 30] loss: 0.271
[Epoch 2, Batch 40] loss: 0.250
[Epoch 2, Batch 50] loss: 0.235
[Epoch 2, Batch 60] loss: 0.235
[Epoch 3, Batch 10] loss: 0.205
[Epoch 3, Batch 20] loss: 0.184
[Epoch 3, Batch 30] loss: 0.179
[Epoch 3, Batch 40] loss: 0.161
[Epoch 3, Batch 50] loss: 0.174
[Epoch 3, Batch 60] loss: 0.172
[Epoch 4, Batch 10] loss: 0.151
[Epoch 4, Batch 20] loss: 0.149
[Epoch 4, Batch 30] loss: 0.133
[Epoch 4, Batch 40] loss: 0.135
[Epoch 4, Batch 50] loss: 0.134
[Epoch 4, Batch 60] loss: 0.136
[Epoch 5, Batch 10] loss: 0.121
[Epoch 5, Batch 20] loss: 0.114
[Epoch 5, Batch 30] loss: 0.118
[Epoch 5, Batch 40] loss: 0.106
[Epoch 5, Batch 50] loss: 0.132
[Epoch 5, Batch 60] loss: 0.103
Finished Training


In [30]:
correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on the test set: {100 * correct / total:.2f}%')

Accuracy on the test set: 96.43%


In [62]:
import gradio as gr
from PIL import Image
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms

# Define a prediction function
def predict_digit(image):
    # Check if the input is a numpy array
    if isinstance(image, np.ndarray):
        # Convert the numpy array to a PIL Image
        image_pil = Image.fromarray(image.astype('uint8')).convert('L')
        # Apply the transformation
        image_tensor = transform(image_pil).unsqueeze(0)  # Add batch dimension
        # Perform the prediction
        with torch.no_grad():
            output = model(image_tensor)
            _, predicted = torch.max(output.data, 1)
        return predicted.item()
    else:
        return "Invalid input format."

# Create the Gradio interface
iface = gr.Interface(
    fn=predict_digit,
    inputs=gr.Sketchpad(),
    outputs=gr.Label(),
    title="MNIST Digit Classifier",
    description="Draw a digit on the canvas and see the model's prediction."
)

iface.launch()

Running on local URL:  http://127.0.0.1:7879

To create a public link, set `share=True` in `launch()`.


In [50]:
# Testing the prediction function with the uploaded image
# Testing the prediction function with the uploaded image
image_path = r'C:\Users\madhu\OneDrive\Desktop\shodh AI\Screenshot 2024-06-12 220434.png'
image = np.array(Image.open(image_path).convert('L'))
prediction = predict_digit(image)
print("Predicted digit:", prediction)

Predicted digit: 7
